In [74]:
is_train = True
basic_epochs = 260
model_layer = 18
local = 1
if (local):
    log_dir = '/home/zyp/w/d2l/classify_leave/log'
    data_dir = '/home/zyp/w/d2l/classify_leave/data'
    history_dir = '/home/zyp/w/d2l/classify_leave/history'
    model_file = '/home/zyp/w/d2l/classify_leave/log/epochs'+str(basic_epochs)+'.model'
else:
    log_dir = '/content/log'
    data_dir = '/content/data'
    history_dir = '/content/history'
    model_file = '/content/history/epochs'+str(basic_epochs)+'.model'
    drive_log_dir = '/content/drive/MyDrive/colab/classify_leave/log'
    drive_model_file = '/content/drive/MyDrive/colab/classify_leave/log/epochs'+str(basic_epochs)+'.model'
    
!mkdir -p $log_dir
!mkdir -p $data_dir
!mkdir -p $history_dir

total_epochs = basic_epochs

record = []

time: 420 ms (started: 2022-05-25 14:49:27 +08:00)


In [4]:
#显示运行时间
!pip install ipython-autotime
%load_ext autotime

# 挂载 drive
from google.colab import drive
drive.mount('/content/drive')

# 准备数据集
!cp /content/drive/MyDrive/colab/classify_leave/classify-leaves.zip /content/data
!unzip /content/data/classify-leaves.zip -d /content/data/
!ls /content/data/images/ | wc -l

!cp $drive_model_file /content/history/

!nvidia-smi

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/data/images/5520.jpg  
  inflating: /content/data/images/5521.jpg  
  inflating: /content/data/images/5522.jpg  
  inflating: /content/data/images/5523.jpg  
  inflating: /content/data/images/5524.jpg  
  inflating: /content/data/images/5525.jpg  
  inflating: /content/data/images/5526.jpg  
  inflating: /content/data/images/5527.jpg  
  inflating: /content/data/images/5528.jpg  
  inflating: /content/data/images/5529.jpg  
  inflating: /content/data/images/553.jpg  
  inflating: /content/data/images/5530.jpg  
  inflating: /content/data/images/5531.jpg  
  inflating: /content/data/images/5532.jpg  
  inflating: /content/data/images/5533.jpg  
  inflating: /content/data/images/5534.jpg  
  inflating: /content/data/images/5535.jpg  
  inflating: /content/data/images/5536.jpg  
  inflating: /content/data/images/5537.jpg  
  inflating: /content/data/images/5538.jpg  
  inflating: /content/data/images/5539.jpg  
  inflating: /content/data/im

In [75]:
import math
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision.models as models
from torchvision.io import read_image
import torchvision.transforms as transforms
import json

time: 560 µs (started: 2022-05-25 14:49:33 +08:00)


In [76]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

time: 1.84 ms (started: 2022-05-25 14:49:35 +08:00)


In [77]:
all_train_label = pd.read_csv(data_dir + '/train.csv')
print(all_train_label)

                  image                    label
0          images/0.jpg         maclura_pomifera
1          images/1.jpg         maclura_pomifera
2          images/2.jpg         maclura_pomifera
3          images/3.jpg         maclura_pomifera
4          images/4.jpg         maclura_pomifera
...                 ...                      ...
18348  images/18348.jpg          aesculus_glabra
18349  images/18349.jpg  liquidambar_styraciflua
18350  images/18350.jpg            cedrus_libani
18351  images/18351.jpg      prunus_pensylvanica
18352  images/18352.jpg          quercus_montana

[18353 rows x 2 columns]
time: 20.8 ms (started: 2022-05-25 14:49:37 +08:00)


In [78]:
leave_class = sorted(list(set(all_train_label['label'])))
class_to_idx = dict(zip(leave_class, range(len(leave_class)))) 
idx_to_class = {v:k for k, v in class_to_idx.items()}

print(list(class_to_idx.items())[0:4])
print(list(idx_to_class.items())[0:4])
print(len(leave_class))

[('abies_concolor', 0), ('abies_nordmanniana', 1), ('acer_campestre', 2), ('acer_ginnala', 3)]
[(0, 'abies_concolor'), (1, 'abies_nordmanniana'), (2, 'acer_campestre'), (3, 'acer_ginnala')]
176
time: 1.69 ms (started: 2022-05-25 14:49:40 +08:00)


In [79]:
class LeaveDataset(Dataset):
    
    all_train_label = pd.read_csv(data_dir + '/train.csv')
    test_label = pd.read_csv(data_dir + '/test.csv')
    offset = math.floor(len(all_train_label) * 0.8)
    train_mode = "train"
    validation_mode = "validation"
    
    def __init__(self, mode):
        self.mode = mode
        
    def __getitem__(self, idx):
        all_train_label = LeaveDataset.all_train_label.copy(deep=False)
        test_label = LeaveDataset.test_label.copy(deep=False)

        if self.mode == LeaveDataset.train_mode:
            img_file, label = all_train_label.iloc[idx][0], all_train_label.iloc[idx][1]
        elif self.mode == LeaveDataset.validation_mode:
            idx = LeaveDataset.offset + idx
            img_file, label = all_train_label.iloc[idx][0], all_train_label.iloc[idx][1]
        else:
            img_file = test_label.iloc[idx][0]
            
        img = read_image(str(data_dir + '/' + img_file))
        if self.mode == LeaveDataset.train_mode:
            t = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomResizedCrop(size=(224, 224), scale=(0.5, 1)),
                transforms.ConvertImageDtype(torch.float),
            ])
        else:
            t = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ConvertImageDtype(torch.float),
            ])         
        
        img = t(img)
        if 'label' in locals():
            label = class_to_idx[label]
            return img, label
        else:
            return img

    def __len__(self):
        test_label = LeaveDataset.test_label.copy(deep=False)

        if self.mode == LeaveDataset.train_mode:
            return LeaveDataset.offset
        elif self.mode == LeaveDataset.validation_mode:
            return len(all_train_label) - LeaveDataset.offset 
        else:
            return len(test_label)

        
training_data = LeaveDataset("train")
validation_data = LeaveDataset("validation")
test_data = LeaveDataset("test")
print(len(training_data), len(validation_data), len(test_data))

train_dataloader = DataLoader(training_data, batch_size=64)
validation_dataloader = DataLoader(validation_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


14682 3671 8800
time: 18.8 ms (started: 2022-05-25 14:49:42 +08:00)


In [80]:
def get_resnet(model_layer, model_file=None):
    pretained = (basic_epochs == 0)
    if model_layer == 18:
        model = models.resnet18(pretrained=pretained)
    else:
        model = models.resnet50(pretrained=pretained)
    
    model.fc = nn.Linear(model.fc.in_features, len(leave_class))
    if pretained:
        nn.init.xavier_uniform_(model.fc.weight)
    else:
        model.load_state_dict(torch.load(model_file, map_location=torch.device(device)))
    return model
    

model = get_resnet(model_layer, model_file=model_file)
print(f'model: restnet{model_layer}, load from:{model_file}')
model.train(is_train)

model = model.to(device)

model: restnet18, load from:/home/zyp/w/d2l/classify_leave/log/epochs260.model
time: 123 ms (started: 2022-05-25 14:49:45 +08:00)


In [81]:
batch_size = 64
epochs = 5

# Initialize the loss function / optimizer
loss_fn = nn.CrossEntropyLoss()

learning_rate = 3e-5
weight_decay = 1e-3
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

time: 736 µs (started: 2022-05-25 14:49:54 +08:00)


In [66]:
for batch, (X, y) in enumerate(train_dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    break



time: 241 ms (started: 2022-05-25 04:54:16 +00:00)


In [67]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    correct = 0
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
    correct /= size
    print(f"[Train] Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss.item():>8f} \n")
    return round(100*correct, 1), round(loss.item(), 6)


def validation_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"[Test] Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return round(100*correct, 1), round(test_loss, 6)


time: 22.1 ms (started: 2022-05-25 04:54:17 +00:00)


In [68]:
validation_loop(validation_dataloader, model, loss_fn)
train_loop(train_dataloader, model, loss_fn, optimizer)

[Test] Accuracy: 90.2%, Avg loss: 0.310235 

[Train] Accuracy: 98.0%, Avg loss: 0.020176 



(98.0, 0.020176)

time: 1min 12s (started: 2022-05-25 04:54:17 +00:00)


In [69]:
def save_state(model, record):
    print("save model at epochs " + str(total_epochs))
    torch.save(model.state_dict(), log_dir+'/epochs' + str(total_epochs) + '.model')
    with open(log_dir + "/acc" + str(total_epochs) + ".log", 'a') as f:
        json.dump(record, f)

time: 2.93 ms (started: 2022-05-25 04:55:29 +00:00)


In [74]:
epochs = 10
print(optimizer)
for t in range(epochs):
    total_epochs+=1
    print(f"Epoch {total_epochs}\n-------------------------------")
    l1, acc1 = train_loop(train_dataloader, model, loss_fn, optimizer)
    l2, acc2 = validation_loop(validation_dataloader, model, loss_fn)
    record.append([total_epochs, l1, acc1, l2, acc2])
    if total_epochs % 20 == 0:
        save_state(model, record)
print("Done!")

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 3e-05
    maximize: False
    weight_decay: 0
)
Epoch 251
-------------------------------
[Train] Accuracy: 99.7%, Avg loss: 0.000291 

[Test] Accuracy: 94.5%, Avg loss: 0.192564 

Epoch 252
-------------------------------
[Train] Accuracy: 99.8%, Avg loss: 0.000476 

[Test] Accuracy: 94.3%, Avg loss: 0.195592 

Epoch 253
-------------------------------
[Train] Accuracy: 99.8%, Avg loss: 0.000378 

[Test] Accuracy: 94.5%, Avg loss: 0.195348 

Epoch 254
-------------------------------
[Train] Accuracy: 99.8%, Avg loss: 0.000427 

[Test] Accuracy: 94.4%, Avg loss: 0.196668 

Epoch 255
-------------------------------
[Train] Accuracy: 99.9%, Avg loss: 0.001136 

[Test] Accuracy: 94.4%, Avg loss: 0.195786 

Epoch 256
-------------------------------
[Train] Accuracy: 99.8%, Avg loss: 0.002857 

[Test] Accuracy: 94.4%, Avg loss: 0.197856 

Epoch 257
-------------------------------
[Train] Accuracy: 99.

In [75]:
!ls -alh $log_dir

!mv /content/log/*.model $drive_log_dir
!mv /content/log/*.log $drive_log_dir

total 44M
drwxr-xr-x 3 root root 4.0K May 25 05:57 .
drwxr-xr-x 1 root root 4.0K May 25 03:28 ..
-rw-r--r-- 1 root root 1.2K May 25 05:57 acc260.log
-rw-r--r-- 1 root root  44M May 25 05:57 epochs260.model
drwxr-xr-x 2 root root 4.0K May 25 03:29 .ipynb_checkpoints
time: 583 ms (started: 2022-05-25 06:47:50 +00:00)


---
## 测试


In [82]:
# 测试集预测
test_pred = torch.tensor([], dtype=torch.int64)

with torch.no_grad():
    for X in test_dataloader:
        X = X.to(device)
        pred = model(X)
        test_pred = torch.hstack((test_pred, pred.argmax(1)))
        
print(test_pred.shape)

test_class = []
for i in enumerate(test_pred.numpy()):
    test_class.append(idx_to_class[i[1]])
print(len(test_class))
test_class2 = pd.DataFrame(test_class, columns=['label'])

result = pd.concat([LeaveDataset.test_label, test_class2], axis=1)
result.to_csv(log_dir + "/result.csv", index=False)
result

torch.Size([8800])
8800


,image,label
0,images/18353.jpg,asimina_triloba
1,images/18354.jpg,populus_grandidentata
2,images/18355.jpg,platanus_acerifolia
3,images/18356.jpg,pinus_bungeana
4,images/18357.jpg,platanus_acerifolia
...,...,...
8795,images/27148.jpg,pinus_thunbergii
8796,images/27149.jpg,crataegus_crus-galli
8797,images/27150.jpg,pinus_thunbergii
8798,images/27151.jpg,juniperus_virginiana


time: 2min 45s (started: 2022-05-25 14:50:19 +08:00)
